necessary packages

In [1]:
import numpy as np
from ipywidgets import interact
import plotly.graph_objects as go
from scipy.special import gamma
from scipy.integrate import quad as integrate

define function for updating the layout of the figure:
* updates the layout of a plotly figure with predefined settings
* the **title** dict includes all title properties: **'x'** specifies the position of the title along the x-axis, **'xanchor'** specifies the anchor point of the title along the x-axis

In [108]:
def update_layout_of_graph(fig: go.Figure, title: str = 'Plot') -> go.Figure:
    fig.update_layout(
        width=1000,
        height=500,
        autosize=False,
        plot_bgcolor='black',
        title={'text': title, 'x': 0.5, 'xanchor': 'center'},  # dict includes all title properties; specifies the position of the title along the x-axis
        xaxis_title='t / ms',
        yaxis_title='U / V',
        legend=dict(yanchor="top", y=1, xanchor="right", x=0.95),
        xaxis=dict(showline=True, linewidth=1, linecolor='black'),
        yaxis=dict(showline=True, linewidth=1, linecolor='black')
    )
    return fig

define function for creating a line plot:
* creates a Plotly scatter trace representing a line plot
* **x_lines** x-coordinates of the points on the line with an empty default array
* **y_lines** y-coordinates of the points on the line with an empty default array

In [3]:
def line_plot(x_lines: np.array = np.array([]), y_lines: np.array = np.array([]), title: str = 'predicted function',
show_legend: bool = True, visible: bool = True, ) -> go.Scatter:

    scatter = go.Scatter(x=x_lines, y=y_lines, line=dict(color="orange", width=3), name=title, showlegend= show_legend, visible=visible)
    return scatter

define function for a scatter plot:
* creates a scatter plot with Plotly
* **x_dots** is an array of x-coordinates for the data points in the scatter plot with an empty default array
* **y_dots** is an array of y-coordinates for the data points in the scatter plot with an empty default array

In [4]:
def scatter_plot(visible: bool = True, x_dots: np.array = np.array([]), y_dots: np.array = np.array([]), name_dots: str = 'observed points',
show_legend: bool = True, color: str = 'red') -> go.Scatter:

    scatter = go.Scatter(x=x_dots, y=y_dots, visible=visible, mode='markers', name=name_dots, marker=dict(color=color, size=8), showlegend=show_legend)
    return scatter

define function for plotting the uncertainty area:
* generates an area plot to represent the uncertainty using the Plotly
* **x_lines** an array of x-coordinates for the lines outlining the area plot with an empty default array
* **y_lower** an array of y-coordinates representing the lower boundary of the uncertainty area plot with an empty default array
* **y_upper** an array of y-coordinates representing the upper boundary of the uncertainty area plot with an empty default array

In [5]:
def uncertainty_area_plot(visible: bool = True, x_lines: np.array = np.array([]), y_lower: np.array = np.array([]), y_upper: np.array = np.array([]), name: str = 'mean plus/minus standard deviation') -> go.Scatter:

    scatter = go.Scatter(x=np.concatenate((x_lines, x_lines[::-1])), y=np.concatenate((y_upper, y_lower[::-1])), visible=visible, fill='toself', fillcolor='rgba(189, 195, 199, 0.5)', line=dict(color='rgba(200, 200, 200, 0)'), hoverinfo='skip', showlegend=True, name=name)
    return scatter

define squared exponential kernel:
* **variable1** & **variable2** correspond to t & t'
* **sigma** describes the average distance away from the function mean and l determines the reach of influence on neighbors
* **sigma** & **length** is assigned the float standard value 1
* **variable_1** & **variable_2** expect an array

In [237]:
class SquaredExponentialKernel:
    def __init__(self, sigma: float = 1, length: float = 1, alpha: float = 0.75, beta: float = 0.75):
        self.sigma = sigma
        self.length = length
        self.alpha = alpha
        self.beta = beta

    def __call__(self, tau_1: np.array, tau_2: np.array):
        def kernel(variable_1, variable_2):
            k_SE = float(self.sigma**2 * np.exp(-(np.linalg.norm(variable_1 - variable_2)**2) /(2 * self.length**2)))
            return k_SE
        return kernel(tau_1, tau_2)

visualize the squared exponential kernel:
* only for testing the squared exponential kernel and monitor the effect of changing the parameters
* **@interact** generates a user interface which allows for an interactive regulation of the parameters length, sigma and variable_2 in the defined ranges
* the function **update** is called whenever the values of the sliders are changed
* **batch_update()** ensures that all updates to the figure within the block are batched together

In [7]:
x_lines = np.arange(-10, 10, 0.1)
kernel = SquaredExponentialKernel()  # initialize an instance of the class SquaredExponentialKernel, length and sigma have already standard values
data = line_plot(x_lines=x_lines, y_lines=np.array([kernel(x, 0) for x in x_lines]))  # generates the data for the line distribution

fig = go.FigureWidget(data)  # generates an interactive plot-widget with the passed data
fig = update_layout_of_graph(fig, title='squared exponential kernel')

@interact(length=(0.1, 3, 0.1), sigma=(0.1, 3, 0.1), variable_2=(-10, 10, 0.1))
def update(length=1, sigma=1, variable_2=0):
    with fig.batch_update():
        kernel = SquaredExponentialKernel(sigma=sigma, length=length)  # a new instance of the class SquaredExponentialKernel with the updated length and sigma value is generated
        fig.data[0].y = np.array([kernel(x, variable_2) for x in x_lines])  # updates the y-values plot data, data can contain multiply scatter plots
        # with data[0].y the y-coordinates of the first scatter plot are accessed

fig

interactive(children=(FloatSlider(value=1.0, description='length', max=3.0, min=0.1), FloatSlider(value=1.0, d…

FigureWidget({
    'data': [{'line': {'color': 'orange', 'width': 3},
              'name': 'predicted function',
              'showlegend': True,
              'type': 'scatter',
              'uid': '1b9b08c9-eebc-4a5e-a2c5-e39ff4484a4a',
              'visible': True,
              'x': array([-1.00000000e+01, -9.90000000e+00, -9.80000000e+00, -9.70000000e+00,
                          -9.60000000e+00, -9.50000000e+00, -9.40000000e+00, -9.30000000e+00,
                          -9.20000000e+00, -9.10000000e+00, -9.00000000e+00, -8.90000000e+00,
                          -8.80000000e+00, -8.70000000e+00, -8.60000000e+00, -8.50000000e+00,
                          -8.40000000e+00, -8.30000000e+00, -8.20000000e+00, -8.10000000e+00,
                          -8.00000000e+00, -7.90000000e+00, -7.80000000e+00, -7.70000000e+00,
                          -7.60000000e+00, -7.50000000e+00, -7.40000000e+00, -7.30000000e+00,
                          -7.20000000e+00, -7.10000000e+00, -7.000000

functions for creating the covariance matrix, each of them were implemented separately in such a way as to make them comprehensible with the notes, which is why they are referenced:
* **val_1** & **val_2** are equal to a certain element of the arrays **t** & **t'**
* **alpha**, **R** & **C** are set to a specific value

In [8]:
# test

In [9]:
def k_fu(val_1, val_2, cov_func, R=0.87, C=330e-6, L=1e-3):
    alpha = cov_func.alpha
    beta = cov_func.beta
    l = cov_func.length

    c = R * C * cov_func(val_1, val_2)

    def integrand_a(tau):
        a = -1 / (gamma(1+alpha) * l**2)
        return a * (tau - val_2) * (val_1 - tau)**alpha * cov_func(tau, val_2)

    def integrand_b(tau):
        b = -1 / (gamma(1+alpha+beta) * l**2)
        return b * (tau - val_2) * (val_1 - tau)**(alpha+beta) * cov_func(tau, val_2)

    integral_val_a, _ = integrate(integrand_a, 0, val_1, epsabs=1e-9, epsrel=1e-9, limit=10000)
    integral_val_b, _ = integrate(integrand_b, 0, val_1, epsabs=1e-9, epsrel=1e-9, limit=10000)
    return integral_val_a + R / L * integral_val_b + c

In [10]:
def k_uf(val_1, val_2, cov_func, R=0.87, C=330e-6, L=1e-3):
    alpha = cov_func.alpha
    beta = cov_func.beta
    l = cov_func.length

    c = R * C * cov_func(val_1, val_2)

    def integrand_a(tau):
        a = 1 / (gamma(1+alpha) * l**2)
        return a * (val_1 - tau) * (val_2 - tau)**alpha * cov_func(val_1, tau)

    def integrand_b(tau):
        b = 1 / (gamma(1+alpha+beta) * l**2)
        return b * (val_1 - tau) * (val_2 - tau)**(alpha+beta) * cov_func(val_1, tau)

    integral_val_a, _ = integrate(integrand_a, 0, val_2, epsabs=1e-9, epsrel=1e-9, limit=10000)
    integral_val_b, _ = integrate(integrand_b, 0, val_2, epsabs=1e-9, epsrel=1e-9, limit=10000)
    return integral_val_a + R / L * integral_val_b + c

In [11]:
def k_uu(val_1, val_2, cov_func):
    return cov_func(val_1, val_2)

In [12]:
def k_ff(val_1, val_2, cov_func, R=0.87, C=330e-6, L=1e-3):
    alpha = cov_func.alpha
    beta = cov_func.beta
    l = cov_func.length

    # simple expressions, see notes:
    a = R * C * k_uf(val_1, val_2, cov_func, R, C, L)
    b = R * C * (k_fu(val_1, val_2, cov_func, R, C, L) - R * C * k_uu(val_1, val_2, cov_func))

    # more complex integral c, see notes:
    def c():
        def A(val_1, val_2, tau):
            a = 1/gamma(1+alpha) * (val_2 - tau)**alpha * 1/l**2 * (val_1 - tau)
            return a * cov_func(val_1, tau)

        def B(val_1, val_2):
            integrand = lambda tau: A(val_1, val_2, tau)
            integral_val, _ = integrate(integrand, 0, val_2, epsabs=1e-8, epsrel=1e-8, limit=10000)
            return integral_val

        def derivative_B(tau, val_2, h= 0.01*np.sqrt(2e-53)):
            def B_tau(tau):
                return B(tau, val_2)
            dB = (B_tau(tau + h) - B_tau(tau - h)) / (2*h)
            return dB

        def double_integral(val_1, val_2):
            integrand = lambda tau: 1/gamma(1 + alpha) * (val_1 - tau)**alpha * derivative_B(tau, val_2)
            integral_val, _ = integrate(integrand, 0, val_1, epsabs=1e-8, epsrel=1e-8, limit=10000)
            return integral_val

        c = double_integral(val_1, val_2)
        return c

    # more complex integral d, see notes:
    def d():
        def A(val_1, val_2, tau):
            a = R / (L * gamma(1+alpha+beta)) * (val_2 - tau)**(alpha+beta) * 1/l**2 * (val_1 - tau)
            return a * cov_func(val_1, tau)

        def B(val_1, val_2):
            integrand = lambda tau: A(val_1, val_2, tau)
            integral_val, _ = integrate(integrand, 0, val_2, epsabs=1e-8, epsrel=1e-8, limit=10000)
            return integral_val

        def derivative_B(tau, val_2, h= 0.01*np.sqrt(2e-53)):
            def B_tau(tau):
                return B(tau, val_2)
            dB = (B_tau(tau + h) - B_tau(tau - h)) / (2*h)
            return dB

        def double_integral(val_1, val_2):
            integrand = lambda tau: 1/gamma(1 + alpha) * (val_1 - tau)**alpha * derivative_B(tau, val_2)
            integral_val, _ = integrate(integrand, 0, val_1, epsabs=1e-8, epsrel=1e-8, limit=10000)
            return integral_val

        d = double_integral(val_1, val_2)
        return d

    # more complex integral e, see notes:
    def e():
        def A(val_1, val_2, tau):
            a = 1/gamma(1+alpha) * (val_2 - tau)**alpha * 1/l**2 * (val_1 - tau)
            return a * cov_func(val_1, tau)

        def B(val_1, val_2):
            integrand = lambda tau: A(val_1, val_2, tau)
            integral_val, _ = integrate(integrand, 0, val_2, epsabs=1e-8, epsrel=1e-8, limit=10000)
            return integral_val

        def derivative_B(tau, val_2, h= 0.01*np.sqrt(2e-53)):
            def B_tau(tau):
                return B(tau, val_2)
            dB = (B_tau(tau + h) - B_tau(tau - h)) / (2*h)
            return dB

        def double_integral(val_1, val_2):
            integrand = lambda tau: 1 / gamma(1 + alpha + beta) * (val_1 - tau)**(alpha + beta) * derivative_B(tau, val_2)
            integral_val, _ = integrate(integrand, 0, val_1, epsabs=1e-8, epsrel=1e-8, limit=10000)
            return integral_val

        e = R / L * double_integral(val_1, val_2)
        return e

    # more complex integral f, see notes:
    def f():
        def A(val_1, val_2, tau):
            a = R / (L * gamma(1 + alpha + beta)) * (val_2 - tau)**(alpha + beta) * 1/l**2 * (val_1 - tau)
            return a * cov_func(val_1, tau)

        def B(val_1, val_2):
            integrand = lambda tau: A(val_1, val_2, tau)
            integral_val, _ = integrate(integrand, 0, val_2, epsabs=1e-8, epsrel=1e-8, limit=10000)
            return integral_val

        def derivative_B(tau, val_2, h= 0.01*np.sqrt(2e-53)):
            def B_tau(tau):
                return B(tau, val_2)
            dB = (B_tau(tau + h) - B_tau(tau - h)) / (2*h)
            return dB

        def double_integral(val_1, val_2):
            integrand = lambda tau: 1 / gamma(1 + alpha + beta) * (val_1 - tau)**(alpha + beta) * derivative_B(tau, val_2)
            integral_val, _ = integrate(integrand, 0, val_1, epsabs=1e-8, epsrel=1e-8, limit=10000)
            return integral_val

        d = R / L * double_integral(val_1, val_2)
        return d

    return a + b + c() + d() + e() + f()

testing the implemented functions above:

In [13]:
val_1 = 0.1
val_2 = 0.1
x = np.arange(0.01, 3, 0.01)
#x = np.array([0.3, 0.8, 1.4, 1.8, 2]) please
cov_func = SquaredExponentialKernel()

for i in range(len(x)):
    kuf = k_uf(x[i], x[i], cov_func)
    kfu = k_fu(x[i], x[i], cov_func)
    kuu = k_uu(x[i], x[i], cov_func)
    kff = k_ff(x[i], x[i], cov_func)
    print('kuf = ', kuf, 'kuu = ', kuu, 'kfu = ', kfu, 'kff = ', kff)

kuf =  0.0003070494106657661 kuu =  1.0 kfu =  0.0003070494106657661 kff =  9.388136160428288e-08
kuf =  0.0005070424443475588 kuu =  1.0 kfu =  0.0005070424443475588 kff =  2.0871736154436826e-07
kuf =  0.001186970645058045 kuu =  1.0 kfu =  0.001186970645058045 kff =  5.991321343923294e-07
kuf =  0.002735931327708859 kuu =  1.0 kfu =  0.002735931327708859 kff =  1.4885453583704266e-06
kuf =  0.005613947511904068 kuu =  1.0 kfu =  0.005613947511904068 kff =  3.141102251335316e-06
kuf =  0.010341646737587954 kuu =  1.0 kfu =  0.010341646737587954 kff =  5.855747146723003e-06
kuf =  0.017493208442374654 kuu =  1.0 kfu =  0.017493208442374654 kff =  9.962173877611525e-06
kuf =  0.02769105925971123 kuu =  1.0 kfu =  0.02769105925971123 kff =  1.581777981692619e-05
kuf =  0.04160163276631617 kuu =  1.0 kfu =  0.04160163276631617 kff =  2.3805231124418744e-05
kuf =  0.05993183223639681 kuu =  1.0 kfu =  0.05993183223639681 kff =  3.433043166013905e-05
kuf =  0.08342598450086934 kuu =  1.0 k

a function for creating the sub and covariance matrix as described in the notes:
* works only correctly for squared matrices [n1 x n2] where n1 = n2

In [14]:
def create_sub_matrix(variable_1, variable_2, specified_cov_function, cov_function):
    n1 = len(variable_1)
    n2 = len(variable_2)
    sub_matrix = np.zeros((n1, n2))

    for i, val_1 in enumerate(variable_1):
        for j, val_2 in enumerate(variable_2):
            sub_matrix[i, j] = specified_cov_function(val_1, val_2, cov_function)

    return sub_matrix

a function for creating the covariance matrix as described in the notes:
* works only correctly for squared matrices [2*n1 x 2*n2] where n1 = n2

In [15]:
def create_cov_matrix(variable_1, variable_2, cov_function):
    n1 = len(variable_1)
    n2 = len(variable_2)
    cov = np.zeros((2*n1, 2*n2))

    kuu_sub_matrix = create_sub_matrix(variable_1, variable_2, k_uu, cov_function)
    kuf_sub_matrix = create_sub_matrix(variable_1, variable_2, k_uf, cov_function)
    kfu_sub_matrix = create_sub_matrix(variable_1, variable_2, k_fu, cov_function)
    kff_sub_matrix = create_sub_matrix(variable_1, variable_2, k_ff, cov_function)

    for i in range(2*n1):
        for j in range(2*n2):

            if i <= n1 - 1 and j <= n2 - 1:
                #print('kuu')
                cov[i, j] = kuu_sub_matrix[i, j]
            elif i <= n1 - 1 and n2 <= j <= 2*n2 - 1:
                #print('kuf')
                cov[i, j] = kuf_sub_matrix[i, j - n2]
            elif n1 <= i <= 2*n1 - 1 and j <= n2 - 1:
                #print('kfu')
                cov[i, j] = kfu_sub_matrix[i - n1, j]
            else:
                #print('kff')
                cov[i, j] = kff_sub_matrix[i - n1, j - n2]

    return cov

testing the implemented functions for creating the covariance matrix above

In [16]:
variable_1 = [1, 1, 1, 1]
variable_2 = [1, 1, 1, 1]
#variable_1 = np.array([0.3, 0.8, 1.4, 1.8, 2])
#variable_2 = np.array([0.3, 0.8, 1.4, 1.8, 2])

cov_func = SquaredExponentialKernel()
cov = create_cov_matrix(variable_1, variable_2, cov_func)

print('covariance matrix = ', cov)

covariance matrix =  [[1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.37400292e+02 1.37400292e+02 1.37400292e+02 1.37400292e+02]
 [1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.37400292e+02 1.37400292e+02 1.37400292e+02 1.37400292e+02]
 [1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.37400292e+02 1.37400292e+02 1.37400292e+02 1.37400292e+02]
 [1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.37400292e+02 1.37400292e+02 1.37400292e+02 1.37400292e+02]
 [1.37400292e+02 1.37400292e+02 1.37400292e+02 1.37400292e+02
  7.88951653e-02 7.88951653e-02 7.88951653e-02 7.88951653e-02]
 [1.37400292e+02 1.37400292e+02 1.37400292e+02 1.37400292e+02
  7.88951653e-02 7.88951653e-02 7.88951653e-02 7.88951653e-02]
 [1.37400292e+02 1.37400292e+02 1.37400292e+02 1.37400292e+02
  7.88951653e-02 7.88951653e-02 7.88951653e-02 7.88951653e-02]
 [1.37400292e+02 1.37400292e+02 1.37400292e+02 1.37400292e+02
  7.88951653e-02 7.88951653e-02 7.88951653

define function for calculating q_u transposed


In [17]:
def q_u_transposed(variable_1, variable_2, cov_function):
    n1 = len(variable_1)
    n2 = len(variable_2)
    q_u_trans = np.zeros((n1, 2*n2))

    kuu_sub_matrix = create_sub_matrix(variable_1, variable_2, k_uu, cov_function)
    kuf_sub_matrix = create_sub_matrix(variable_1, variable_2, k_uf, cov_function)

    for i in range(n1):
        for j in range(2*n2):

            if j <= n2 - 1:
                q_u_trans[i, j] = kuu_sub_matrix[i, j]
            else:
                q_u_trans[i, j] = kuf_sub_matrix[i, j - n2]

    return q_u_trans

In [18]:
def q_f_transposed(variable_1, variable_2, cov_function):
    n1 = len(variable_1)
    n2 = len(variable_2)
    q_f_trans = np.zeros((n1, 2*n2))

    kfu_sub_matrix = create_sub_matrix(variable_1, variable_2, k_fu, cov_function)
    kff_sub_matrix = create_sub_matrix(variable_1, variable_2, k_ff, cov_function)

    for i in range(n1):
        for j in range(2*n2):

            if j <= n2 - 1:
                q_f_trans[i, j] = kfu_sub_matrix[i, j]
            else:
                q_f_trans[i, j] = kff_sub_matrix[i, j - n2]

    return q_f_trans

testing if the function q_u_transposed works properly

In [19]:
variable_1 = [1, 1, 1, 1]
variable_2 = [1, 2, 1, 1]
cov_func = SquaredExponentialKernel()

q_u_trans = q_u_transposed(variable_1, variable_2, cov_func)
print('q_u transposed = ', q_u_trans)

q_u transposed =  [[  1.           0.60653066   1.           1.         137.40029216
  475.66522877 137.40029216 137.40029216]
 [  1.           0.60653066   1.           1.         137.40029216
  475.66522877 137.40029216 137.40029216]
 [  1.           0.60653066   1.           1.         137.40029216
  475.66522877 137.40029216 137.40029216]
 [  1.           0.60653066   1.           1.         137.40029216
  475.66522877 137.40029216 137.40029216]]


new training method: page 113 in GP for ML

In [220]:
def log_marginal_likelihood(data_x, data_y, cov_func, noise):
    n = len(data_x)
    y = np.concatenate([data_y, data_y])
    y_transposed = y.T
    K = create_cov_matrix(data_x, data_x, cov_func) + noise * np.identity(2*n)
    K_inverse = np.linalg.inv(K)

    a = n/2 * np.log(2*np.pi)  # normalization constant
    b = 1/2 * np.log(np.linalg.norm(K))  # complexity penalty
    c = 1/2 * np.dot(y, np.dot(K_inverse, y_transposed))  # involving observed targets
    log_ml = c + b + a  # is actually the negative log marginal likelihood
    return log_ml

sigma_vals = np.arange(0.2, 0.8, 0.1)
length_vals = np.arange(0.4, 1.5, 0.1)
memory = {'sigma': 1, 'length': 1, 'negative_log_marginal_likelihood': 1000}

data_x = np.array([0.07899200618113111, 0.2114781250747911, 0.7827671875747916, 1.587733705431922, 2.530814062574759, 3.457850892931882])
data_y = np.array([0.5427752, -0.006186476, -0.6994109, -0.4137002, -0.1350385, -0.03762791])

for sigma_index, sigma_val in enumerate(sigma_vals):
    for l_index, length_val in enumerate(length_vals):
        cov_func = SquaredExponentialKernel(sigma=sigma_val, length=length_val)
        negative_log_marginal_likelihood = log_marginal_likelihood(data_x, data_y, cov_func, noise = 0.01)
        print('likelihood = ', negative_log_marginal_likelihood)

        if negative_log_marginal_likelihood < memory['negative_log_marginal_likelihood']:
            memory['length'] = length_val
            memory['sigma'] = sigma_val
            memory['negative_log_marginal_likelihood'] = negative_log_marginal_likelihood

        print('obtained sigma and l values that minimize the negative log marginal likelihood: ', 'sigma = ', memory['sigma'], 'length = ', memory['length'])

likelihood =  0.24278579638456055
obtained sigma and l values that minimize the negative log marginal likelihood:  sigma =  0.2 length =  0.4
likelihood =  39.952437172966924
obtained sigma and l values that minimize the negative log marginal likelihood:  sigma =  0.2 length =  0.4
likelihood =  21.431670876137904
obtained sigma and l values that minimize the negative log marginal likelihood:  sigma =  0.2 length =  0.4
likelihood =  21.717639525635295
obtained sigma and l values that minimize the negative log marginal likelihood:  sigma =  0.2 length =  0.4
likelihood =  22.7314123087125
obtained sigma and l values that minimize the negative log marginal likelihood:  sigma =  0.2 length =  0.4
likelihood =  23.55545736906645
obtained sigma and l values that minimize the negative log marginal likelihood:  sigma =  0.2 length =  0.4
likelihood =  24.4508123367693
obtained sigma and l values that minimize the negative log marginal likelihood:  sigma =  0.2 length =  0.4
likelihood =  25.

testing the functions above

In [236]:
cov_func = SquaredExponentialKernel(sigma=0.4, length=0.7)
negative_log_marginal_likelihood = log_marginal_likelihood(data_x, data_y, cov_func, noise = 0.01)
print(negative_log_marginal_likelihood)

23.185422958871243


define class GPR:
* **data_x** is the input of the observed points
* **data_y** is the output of the observed points
* **noise** is the noise in the observed data
* **memory** is a variable where the mean, the covariance matrix and the variance is stored
* **3e-7** add this value to the noise in order to ensure stability when inverting the covariance matrix

In [238]:
class GPR:
    def __init__(self, data_x, data_y, cov_func= SquaredExponentialKernel(sigma=memory['sigma'], length=memory['length']), noise=0.01):
        print('sigma memory = ', memory['sigma'])
        print('length memory = ', memory['length'])
        self.data_x = data_x
        self.data_y = np.concatenate([data_y, data_y])
        self.cov_func = cov_func
        self.noise = noise
        self.memory = None

        n = len(data_x)
        self.inv_cov_matrix_of_input_data = np.linalg.inv(create_cov_matrix(data_x, data_x, cov_func) + (noise + 3e-7)**2 * np.identity(2*n))

    def predict(self, at_values):
        q_u_trans = q_u_transposed(at_values, self.data_x, self.cov_func)
        q_f_trans = q_f_transposed(at_values, self.data_x, self.cov_func)

        # methode paper:
        mean_at_values_u = np.dot(q_u_trans, np.dot(self.data_y, self.inv_cov_matrix_of_input_data.T).T).flatten()
        #mean_at_values_f = np.dot(q_f_trans, np.dot(self.data_y, self.inv_cov_matrix_of_input_data.T).T).flatten()

        covariance_matrix_u = create_sub_matrix(at_values, at_values, k_uu, self.cov_func) - np.dot(q_u_trans, np.dot(self.inv_cov_matrix_of_input_data, q_u_trans.T))
        #covariance_matrix_f = create_sub_matrix(at_values, at_values, k_ff, self.cov_func) - np.dot(q_f_trans, np.dot(self.inv_cov_matrix_of_input_data, q_f_trans.T))

        variance_u = np.abs(np.diag(covariance_matrix_u))
        #variance_f = np.abs(np.diag(covariance_matrix_f))

        self.memory = {'mean': mean_at_values_u, 'covariance_matrix':covariance_matrix_u, 'variance': variance_u}

        return mean_at_values_u

defining function for plotting the GPR:
* **data_x** is a list of observed input data points
* **data_y** is a list of observed output data points
* **x** is equivalent to **at_values** in the GPR, the values we want to do predictions on

In [239]:
def plot_GPR(data_x, data_y, model, x, data_x_mv, data_y_mv, visible=True) -> list:
    mean = model.predict(x)
    standard_deviation = np.sqrt(model.memory['variance'])
    data = []

    for i in range(1, 4):
        data.append(uncertainty_area_plot(x_lines=x, y_lower=mean -i * standard_deviation, y_upper=mean + i*standard_deviation, name=f'mean plus/minus {i}*standard deviation', visible=visible))

    data.append(scatter_plot(x_dots=data_x_mv, y_dots=data_y_mv, visible=visible, color='blue'))
    data.append(scatter_plot(x_dots=data_x, y_dots=data_y, visible=visible, color='red'))
    data.append(line_plot(x_lines=x, y_lines=mean, visible=visible))
    return data

initializing the training data

In [240]:
x_values = np.array([0.07899200618113111, 0.2114781250747911, 0.7827671875747916, 1.587733705431922, 2.530814062574759, 3.457850892931882])
y_values = np.array([0.5427752, -0.006186476, -0.6994109, -0.4137002, -0.1350385, -0.03762791])

x_values_mv = np.array([0.0, 0.019748074787471138, 0.03949605191869113, 0.05924402904991112, 0.07899200618113111, 0.09873998331235112, 0.1184879604435711, 0.1382359375747911, 0.1528843750747911, 0.1675328125747911, 0.1772984375747911, 0.1870640625747911, 0.1968296875747911, 0.20171250007479108, 0.20659531257479108, 0.2114781250747911, 0.2163609375747911, 0.2212437500747911, 0.23100937507479108, 0.24077500007479122, 0.2554234375747911, 0.2700718750747911, 0.29169575900336264, 0.313319642931934, 0.3349435268605055, 0.356567410789077, 0.3781912947176484, 0.3998151786462199, 0.4214390625747913, 0.4472482143605056, 0.4730573661462199, 0.49886651793193426, 0.5246756697176486, 0.5504848215033629, 0.5762939732890773, 0.6021031250747915, 0.6202392857890772, 0.6383754465033629, 0.6565116072176487, 0.6746477679319344, 0.6927839286462202, 0.7109200893605059, 0.7290562500747916, 0.7437046875747916, 0.7534703125747916, 0.7632359375747916, 0.7681187500747916, 0.7730015625747916, 0.7778843750747916, 0.7827671875747916, 0.7876500000747916, 0.7925328125747916, 0.7974156250747916, 0.8071812500747916, 0.8169468750747917, 0.8315953125747917, 0.8462437500747917, 0.8678676340033631, 0.8894915179319345, 0.9111154018605058, 0.9327392857890772, 0.9543631697176487, 0.9759870536462201, 0.9976109375747914, 1.029697991146219, 1.0617850447176471, 1.093872098289075, 1.125959151860503, 1.1580462054319312, 1.190133259003359, 1.222220312574787, 1.2661656250747861, 1.3101109375747848, 1.354056250074784, 1.398001562574783, 1.4419468750747821, 1.485892187574781, 1.52983750007478, 1.587733705431922, 1.645629910789063, 1.703526116146205, 1.761422321503347, 1.8193185268604881, 1.87721473221763, 1.935110937574771, 2.0104457590033413, 2.085780580431911, 2.161115401860481, 2.236450223289051, 2.3117850447176207, 2.387119866146191, 2.46245468757476, 2.530814062574759, 2.599173437574757, 2.667532812574756, 2.735892187574755, 2.8042515625747533, 2.872610937574752, 2.94097031257475, 3.005841964360463, 3.070713616146176, 3.135585267931889, 3.200456919717602, 3.2653285715033147, 3.330200223289028, 3.39507187507474, 3.457850892931882, 3.520629910789024, 3.583408928646165, 3.646187946503307, 3.708966964360448, 3.77174598221759, 3.834525000074731, 3.8959089286461577, 3.957292857217585, 4.0186767857890136, 4.0800607143604415, 4.141444642931868, 4.202828571503295, 4.264212500074722, 4.324898884003292, 4.385585267931861, 4.446271651860433, 4.506958035789003, 4.567644419717572, 4.628330803646143, 4.689017187574713, 4.720406696503284, 4.751796205431854, 4.783185714360426, 4.814575223288997, 4.845964732217565, 4.877354241146137, 4.908743750074708, 4.913626562574708, 4.996634375074707, 5.0])  # x data for model validation

y_values_mv = np.array([1.0, 0.8760085, 0.7584742, 0.6473965, 0.5427752, 0.4446104, 0.3529021, 0.2676504, 0.2073551, 0.1499282, 0.1131847, 0.07763808, 0.04325852, 0.0264972, 0.01001672, -0.006186476, -0.0221159, -0.03777504, -0.06829616, -0.09777685, -0.1401047, -0.1802375, -0.2350784, -0.2861547, -0.3334663, -0.3770134, -0.4167958, -0.4528137, -0.485067, -0.5195908, -0.5507508, -0.5785469, -0.6029792, -0.6240477, -0.6417523, -0.656093, -0.6649102, -0.6726904, -0.6794335, -0.6851396, -0.6898088, -0.6934408, -0.6960359, -0.6976341, -0.6984078, -0.698957, -0.6991497, -0.699289, -0.6993757, -0.6994109, -0.6993953, -0.6993298, -0.6992153, -0.6988424, -0.6982832, -0.6971089, -0.6955508, -0.6924582, -0.6887699, -0.684486, -0.6796065, -0.6741314, -0.6680607, -0.6613944, -0.6506511, -0.6393625, -0.6275285, -0.6151493, -0.6022247, -0.5887548, -0.5747396, -0.5552004, -0.5356877, -0.5162017, -0.4967424, -0.4773097, -0.4579036, -0.4385242, -0.4137002, -0.3897431, -0.366653, -0.3444299, -0.3230737, -0.3025845, -0.2829622, -0.2591426, -0.2368111, -0.2159677, -0.1966124, -0.1787452, -0.1623661, -0.1474751, -0.1350385, -0.1234255, -0.1126361, -0.1026703, -0.09352815, -0.08520957, -0.07771459, -0.07115044, -0.06503115, -0.05935674, -0.0541272, -0.04934253, -0.04500273, -0.0411078, -0.03762791, -0.03438695, -0.03138492, -0.02862183, -0.02609767, -0.02381245, -0.02176615, -0.01991911, -0.01819999, -0.0166088, -0.01514552, -0.01381017, -0.01260273, -0.01152322, -0.01053854, -0.009622747, -0.008775851, -0.00799785, -0.007288745, -0.006648535, -0.00607722, -0.005795745, -0.005525807, -0.005267404, -0.005020537, -0.004785206, -0.00456141, -0.004349151, -0.004316826, -0.00380201, -0.003782425])  # y data for model validation

print(len(x_values_mv), len(y_values_mv))

x = np.arange(0.01, 5, 0.01)

137 137


plot the output of the GPR

In [234]:
model = GPR(x_values, y_values)
data = plot_GPR(data_x=x_values, data_y=y_values, x=x, model=model, data_x_mv=x_values_mv, data_y_mv=y_values_mv)
fig4 = go.Figure(data)
fig4 = update_layout_of_graph(fig=fig4, title=f'GPR with length {model.cov_func.length}, sigma {model.cov_func.sigma}, alpha {cov_func.alpha}, beta {model.cov_func.beta} and noise {model.noise}')
fig4.show()
fig4.write_html('/Users/simonjamnik/PycharmProjects/GaussianProcess/RLC_voltage_aperiodic_0.75.html')

sigma memory =  0.4000000000000001
length memory =  0.6
